In [1]:
import os
import sys
sys.path.append('../input/pytorch-images-seresnet')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
import albumentations
from albumentations import *
from albumentations.pytorch import ToTensorV2
import gc


import timm

from torch.cuda.amp import autocast, GradScaler

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

IMAGE_SIZE = 640
BATCH_SIZE = 128
TRAIN_PATH = '../input/ranzcr-clip-catheter-line-classification/train'
RESNET_PATH = '../input/kaggle-ranzor-pths/KAGGLE_RANZOR_pth/resnet200d_320_chestx.pth'
SERESNET_PATH = '../input/kaggle-ranzor-pths/KAGGLE_RANZOR_pth/seresnet152d_320_CV96.15.pth'
EFFNET_PATH = '../input/kaggle-ranzor-pths/KAGGLE_RANZOR_pth/tf_efficientnet_b5_ns_CV96.21.pth'
DENSENET_PATH = '../input/kaggle-ranzor-pths/KAGGLE_RANZOR_pth/densenet121_chestx.pth'
INCEPTIONNET_PATH = '../input/kaggle-ranzor-pths/KAGGLE_RANZOR_pth/inception_v3_chestx.pth'

train = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')

class Dataset(Dataset):
    def __init__(self, df, path, transform=None):
        self.df = df
        self.path = path
        self.file_names = df['StudyInstanceUID'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{self.path}/{file_name}.jpg'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image
    
def get_transforms():
        return Compose([
            Resize(IMAGE_SIZE, IMAGE_SIZE),
            Normalize(
            ),
            ToTensorV2(),
        ])
    
def inference(models, train_loader, device):
    features_outputs=[] 
    tk0 = tqdm(enumerate(train_loader), total=len(train_loader))    
    for i, (images) in tk0:
        images = images.to(device)
        for model in models:          
            with torch.no_grad():                
                output = model(images) 
            features_outputs.append(output.cpu().numpy())
    features_outputs = np.concatenate(features_outputs)
    return features_outputs

#train = train[0:128]
train_dataset = Dataset(train, TRAIN_PATH, transform=get_transforms())
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False, 
                         num_workers=4 , pin_memory=True)

torch.save(train_loader, 'train_loader.pth')
gc.collect()
del train_dataset

class ResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d_320'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output
    
model = ResNet200D()
#model.load_state_dict(torch.load(MODEL_PATH,map_location=torch.device('cpu'))['model'])
model.load_state_dict(torch.load(RESNET_PATH)['model'])
model.eval()
models = [model.to(device)]

train_loader = torch.load('./train_loader.pth')
predictions = inference(models, train_loader, device)

features_outputs_Resnet200D = predictions.squeeze()  
np.save('features_outputs_Resnet200D.npy', features_outputs_Resnet200D)
print(features_outputs_Resnet200D.shape)
del features_outputs_Resnet200D
del model
gc.collect()

class SEResNet152D(nn.Module):
    def __init__(self, model_name='seresnet152d'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output
    
model = SEResNet152D()
#model.load_state_dict(torch.load(MODEL_PATH,map_location=torch.device('cpu'))['model'])
model.load_state_dict(torch.load(SERESNET_PATH)['model'])
model.eval()
models = [model.to(device)]

train_loader = torch.load('./train_loader.pth')
predictions = inference(models, train_loader, device)

features_outputs_SEResNet152D = predictions.squeeze()  
np.save('features_outputs_SEResNet152D.npy', features_outputs_SEResNet152D)
print(features_outputs_SEResNet152D.shape)
del features_outputs_SEResNet152D
del model
gc.collect()

class EfficientNetB5(nn.Module):
    def __init__(self, model_name='tf_efficientnet_b5_ns'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.classifier.in_features
        self.model.global_pool = nn.Identity()
        self.model.classifier = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.classifier(pooled_features)
        return output
    
model = EfficientNetB5()
#model.load_state_dict(torch.load(MODEL_PATH,map_location=torch.device('cpu'))['model'])
model.load_state_dict(torch.load(EFFNET_PATH)['model'])
model.eval()
models = [model.to(device)]

train_loader = torch.load('./train_loader.pth')
predictions = inference(models, train_loader, device)

features_outputs_EfficientNetB5 = predictions.squeeze()  
np.save('features_outputs_EfficientNetB5.npy', features_outputs_EfficientNetB5)
print(features_outputs_EfficientNetB5.shape)
del features_outputs_EfficientNetB5
del model
gc.collect()

class DenseNet121(nn.Module):
    def __init__(self, model_name='densenet121'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.classifier.in_features
        self.model.global_pool = nn.Identity()
        self.model.classifier = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.classifier(pooled_features)
        return output
    
model = DenseNet121()
#model.load_state_dict(torch.load(MODEL_PATH,map_location=torch.device('cpu'))['model'])
model.load_state_dict(torch.load(DENSENET_PATH)['model'])
model.eval()
models = [model.to(device)]

train_loader = torch.load('./train_loader.pth')
predictions = inference(models, train_loader, device)

features_outputs_DenseNet121 = predictions.squeeze()  
np.save('features_outputs_DenseNet121.npy', features_outputs_DenseNet121)
print(features_outputs_DenseNet121.shape)
del features_outputs_DenseNet121
del model
gc.collect()


  0%|          | 0/30083 [00:00<?, ?it/s]

(30083, 11)


  0%|          | 0/30083 [00:00<?, ?it/s]

(30083, 11)


  0%|          | 0/30083 [00:00<?, ?it/s]

(30083, 11)


  0%|          | 0/30083 [00:00<?, ?it/s]

(30083, 11)


0